In [1]:
from head import opentrons
import pandas as pd
import numpy as np
from scipy.spatial import distance
import warnings
warnings.filterwarnings("ignore")

INFO: Using numpy backend


In [2]:
class Simulator:
    def __init__(self):
        self.domain = np.linspace(-5,5,num=100)
        
    def generate(self, mu, sig):
        scale = 1/(np.sqrt(2*np.pi)*sig)
        return scale*np.exp(-np.power(self.domain - mu, 2.) / (2 * np.power(sig, 2.)))
    
    def process_batch(self, Cb, fname):
        out = []
        for c in Cb:
            out.append(self.generate(*c))
        out = np.asarray(out)
        df = pd.DataFrame(out.T, index=self.domain)
        df.to_excel(fname, engine='openpyxl')
        
        return 
    
    def make_target(self, ct):
        return self.domain, self.generate(*ct)

In [3]:
sim = Simulator()
target = np.array([-2,0.5])
xt, yt = sim.make_target(target)

In [5]:
import fdasrsf as fs

def APdist(point_a, point_b, time):
    curves = np.zeros((len(point_a), 2))
    curves[...,0] = point_a
    curves[...,1] = point_b
    obj = fs.fdawarp(curves, time)
    obj.srsf_align(parallel=True, MaxItr=50)
    dp = fs.efda_distance_phase(obj.qn[...,0], obj.qn[...,1])
    da = fs.efda_distance(obj.qn[...,0], obj.qn[...,1])

    return -(da+dp)

metric_function = lambda f1,f2 : APdist(f1,f2,xt)

In [6]:
Cmu = [-5,5]
Csig = [0.1,3.5]
bounds = [Cmu, Csig]

In [7]:
optim = opentrons.Optimizer(xt, yt, 
                            bounds, 
                            savedir = '../data',
                            batch_size=4,
                            metric = metric_function
                           )


INFO: Getting suggestions for iteration 0
INFO: Iteration : 0, Samples to be made
INFO: 0	[-1.13784736  1.53470987]
INFO: 1	[4.31451809 0.36154547]
INFO: 2	[4.30855942 1.28381931]
INFO: 3	[-3.70713813  0.58616024]
INFO: ==============================


In [8]:
# random iteration
optim.save()
C0 = np.load('../data/0/new_x.npy')
sim.process_batch(C0, '../data/0.xlsx')
optim.update('../data/0.xlsx')
optim.save()
optim.get_current_best()

INFO: Iteration : 1 Best estimate [[-4.05225936  0.46576337]] with predicted score : [[-0.36698332]]


In [9]:
for i in range(1,21):
    # iteration i selection
    optim.suggest_next()
    optim.save()
    # simulate iteration i new_x 
    Ci = np.load('../data/%d/new_x.npy'%i)
    sim.process_batch(Ci, '../data/%d.xlsx'%i)
    optim.update('../data/%d.xlsx'%i)
    optim.save()
    optim.get_current_best()

INFO: Getting suggestions for iteration 1
INFO: Iteration : 1, Samples to be made
INFO: 0	[-4.99  3.5 ]
INFO: 1	[-1.47080521  0.1034    ]
INFO: 2	[-4.99    0.1034]
INFO: 3	[-4.99        1.16119762]
INFO: ==============================
INFO: Iteration : 2 Best estimate [[-1.69164024  1.35604649]] with predicted score : [[-0.58655229]]
INFO: Getting suggestions for iteration 2
INFO: Iteration : 2, Samples to be made
INFO: 0	[-1.95348093  2.33970844]
INFO: 1	[1.49363784 2.47971719]
INFO: 2	[5.         3.45086121]
INFO: 3	[0.42045385 3.5       ]
INFO: ==============================
INFO: Iteration : 3 Best estimate [[-1.52820661  1.41900027]] with predicted score : [[-0.60458126]]
INFO: Getting suggestions for iteration 3
INFO: Iteration : 3, Samples to be made
INFO: 0	[1.3522031  0.58359164]
INFO: 1	[-0.16019135  1.1113122 ]
INFO: 2	[1.52965743 1.54757176]
INFO: 3	[5.     0.1034]
INFO: ==============================
INFO: Iteration : 4 Best estimate [[0.31023717 1.02599317]] with predicte